# color sorting gameplay

### import header files

In [ ]:
#!/usr/bin/env python
# coding: utf-8
import cv2 as cv
import threading
from time import sleep
from dofbot_config import *
import ipywidgets as widgets
from IPython.display import display
from speech_identify_target import identify_GetTarget

### Create instance, initialize parameters

In [ ]:
## Create instance to get target
target = identify_GetTarget()
# Create camera calibration instance
calibration = Arm_Calibration()
# Initialize some parameters
num = 0
dp = []
xy = [90, 135]
msg = {}
threshold = 140
model = "General"
color_list = {}
# Initialize HSV values
color_hsv = {
    "red": ((0, 43, 46), (10, 255, 255)),
    "green": ((35, 43, 46), (77, 255, 255)),
    "blue": ((100, 43, 46), (124, 255, 255)),
    "yellow": ((21, 43, 46), (28, 255, 255))
}
HSV_path = "/home/jetson/dofbot_ws/src/dofbot_color_identify/scripts/HSV_config.txt"
# XYT parameters path
XYT_path = "/home/jetson/dofbot_ws/src/dofbot_color_identify/scripts/XYT_config.txt"

try:
    read_HSV(HSV_path, color_hsv)
except Exception:
    print("Read HSV_config Error !!!")

try:
    xy, thresh = read_XYT(XYT_path)
except Exception:
    print("Read XYT_config Error !!!")

### Initialize robotic arm position


In [ ]:
import Arm_Lib
# Create robotic arm drive instance
import time
arm = Arm_Lib.Arm_Device()
joints_0 = [xy[0], xy[1], 0, 0, 90, 30]
arm.Arm_serial_servo_write6_array(joints_0, 1000)


### Create widgets

In [ ]:
button_layout = widgets.Layout(width='320px', height='60px', align_self='center')
output = widgets.Output()
# Adjust sliders
joint1_slider = widgets.IntSlider(description='joint1 :', value=xy[0], min=70, max=110, step=1, orientation='horizontal')
joint2_slider = widgets.IntSlider(description='joint2 :', value=xy[1], min=115, max=155, step=1, orientation='horizontal')
threshold_slider = widgets.IntSlider(description='threshold :', value=threshold, min=0, max=255, step=1, orientation='horizontal')

# Enter calibration mode
calibration_model = widgets.Button(description='calibration_model', button_style='primary', layout=button_layout)
calibration_ok = widgets.Button(description='calibration_ok', button_style='success', layout=button_layout)
calibration_cancel = widgets.Button(description='calibration_cancel', button_style='danger', layout=button_layout)
# Choose grab color
color_list_one = widgets.Dropdown(options=['red', 'green', 'blue', 'yellow', 'none'], value='none', disabled=False)
color_list_two = widgets.Dropdown(options=['red', 'green', 'blue', 'yellow', 'none'], value='none', disabled=False)
color_list_three = widgets.Dropdown(options=['red', 'green', 'blue', 'yellow', 'none'], value='none', disabled=False)
color_list_four = widgets.Dropdown(options=['red', 'green', 'blue', 'yellow', 'none'], value='none', disabled=False)
# Target detection grab
target_detection = widgets.Button(description='target_detection', button_style='info', layout=button_layout)
reset_color_list = widgets.Button(description='reset_color_list', button_style='info', layout=button_layout)
grap = widgets.Button(description='grap', button_style='success', layout=button_layout)
# Exit
exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)
imgbox = widgets.Image(format='jpg', height=480, width=640, layout=widgets.Layout(align_self='center'))
color_down = widgets.HBox([exit_button], layout=widgets.Layout(align_self='center'))
color_img = widgets.VBox([imgbox, color_down], layout=widgets.Layout(align_self='center'))
color_identify = widgets.VBox(
    [joint1_slider, joint2_slider, threshold_slider, calibration_model, calibration_ok, calibration_cancel,
     color_list_one, color_list_two, color_list_three, color_list_four, target_detection, grap],
    layout=widgets.Layout(align_self='center'))
controls_box = widgets.HBox([color_img], layout=widgets.Layout(align_self='center'))


### Calibration Callback

In [ ]:
def calibration_model_Callback(value):
    global model
    model = 'Calibration'
    with output: print(model)
def calibration_OK_Callback(value):
    global model
    model = 'calibration_OK'
    with output: print(model)
def calibration_cancel_Callback(value):
    global model
    model = 'calibration_Cancel'
    with output: print(model)
calibration_model.on_click(calibration_model_Callback)
calibration_ok.on_click(calibration_OK_Callback)
calibration_cancel.on_click(calibration_cancel_Callback)

### Color Selection Sequence

In [ ]:

def color_list_one_Callback(value):
    global model,color_list
    model="General"
    if not isinstance(value['new'],str):return
    if value['new'] == "none":
        if '1' in color_list:del color_list['1']
    elif value['new'] == "red":
        color_list['1'] = "red"
    elif value['new']== "green":
        color_list['1'] = "green"
    elif value['new'] == "blue":
        color_list['1'] = "blue"
    elif value['new'] == "yellow":
        color_list['1'] = "yellow"
    with output:
        print("color_list_three_Callback clicked.",color_list)
def color_list_two_Callback(value):
    global model,color_list
    model="General"
    if not isinstance(value['new'],str):return
    if value['new'] == "none":
        if '2' in color_list:del color_list['2']
    elif value['new'] == "red":
        color_list['2'] = "red"
    elif value['new'] == "green":
        color_list['2'] = "green"
    elif value['new'] == "blue":
        color_list['2'] = "blue"
    elif value['new'] == "yellow":
        color_list['2'] = "yellow"
    with output:
        print("color_list_three_Callback clicked.",color_list)
def color_list_three_Callback(value):
    global model,color_list
    model="General"
    if not isinstance(value['new'],str):return
    if value['new'] == "none":
        if '3' in color_list:del color_list['3']
    elif value['new'] == "red":
        color_list['3'] = "red"
    elif value['new'] == "green":
        color_list['3'] = "green"
    elif value['new'] == "blue":
        color_list['3'] = "blue"
    elif value['new'] == "yellow":
        color_list['3'] = "yellow"
    with output:
        print("color_list_three_Callback clicked.",color_list)
def color_list_four_Callback(value):
    global model,color_list
    model="General"
    if not isinstance(value['new'],str):return
    if value['new'] == "none":
        if '4' in color_list:del color_list['4']
    elif value['new'] == "red":
        color_list['4'] = "red"
    elif value['new'] == "green":
        color_list['4'] = "green"
    elif value['new'] == "blue":
        color_list['4'] = "blue"
    elif value['new'] == "yellow":
        color_list['4'] = "yellow"
    with output:
        print("color_list_four_Callback clicked.",color_list)
color_list_one.observe(color_list_one_Callback)
color_list_two.observe(color_list_two_Callback)
color_list_three.observe(color_list_three_Callback)
color_list_four.observe(color_list_four_Callback)

### Grasping Control

In [ ]:

def target_detection_Callback(value):
    global model
    model = 'Detection'
    with output: print(model)
def reset_color_list_Callback(value):
    global model
    model = 'Reset_list'
    with output: print(model)
def grap_Callback(value):
    global model
    model = 'Grap'
    with output: print(model)
def exit_button_Callback(value):
    global model
    model = 'Exit'
    with output: print(model)
target_detection.on_click(target_detection_Callback)
reset_color_list.on_click(reset_color_list_Callback)
grap.on_click(grap_Callback)
exit_button.on_click(exit_button_Callback)

# Load Speech Recognition and Speech Synthesis Libraries

In [ ]:
import smbus
bus = smbus.SMBus(1)

i2c_addr = 0x0f   #Speech recognition module address
asr_add_word_addr  = 0x01   #Entry add address
asr_mode_addr  = 0x02   #Recognition mode setting address, the value is 0-2, 0: cyclic recognition mode 1: password mode, 2: button mode, the default is cyclic detection
asr_rgb_addr = 0x03   #RGB lamp setting address, need to send two bits, the first one is directly the lamp number 1: blue 2: red 3: green
                      #The second byte is brightness 0-255, the larger the value, the higher the brightness
asr_rec_gain_addr  = 0x04    #Identification sensitivity setting address, sensitivity can be set to 0x00-0x7f, the higher the value, the easier it is to detect but the easier it is to misjudge
                             #It is recommended to set the value to 0x40-0x55, the default value is 0x40
asr_clear_addr = 0x05   #Clear the operation address of the power-off cache, clear the cache area information before entering the information
asr_key_flag = 0x06  #Used in key mode, set the startup recognition mode
asr_voice_flag = 0x07   #Used to set whether to turn on the recognition result prompt sound
asr_result = 0x08  #Recognition result storage address
asr_buzzer = 0x09  #Buzzer control register, 1 bit is on, 0 bit is off
asr_num_cleck = 0x0a #Check the number of entries
asr_vession = 0x0b #firmware version number
asr_busy = 0x0c #Busy and busy flag

i2c_speech_addr = 0x30   
speech_date_head = 0xfd

# Define Speech Synthesis Parameter Control Functions

In [ ]:
def I2C_WriteBytes(data):
    global i2c_speech_addr
    for ch in data:
        try:
            bus.write_byte(i2c_speech_addr, ch)
            time.sleep(0.01)
        except:
            print("I2C write error")

# Encoding formats
EncodingFormat_Type = {
    'GB2312': 0x00,
    'GBK': 0x01,
    'BIG5': 0x02,
    'UNICODE': 0x03
}

def Speech_text(text, encoding_format):
    text = text.encode('gb2312')
    size = len(text) + 2
    DataHead = speech_date_head
    Length_HH = size >> 8
    Length_LL = size & 0x00ff
    Command = 0x01
    EncodingFormat = encoding_format

    Data_Pack = [DataHead, Length_HH, Length_LL, Command, EncodingFormat]

    I2C_WriteBytes(Data_Pack)
    I2C_WriteBytes(text)

def SetBase(text):
    text = text.encode('gb2312')
    size = len(text) + 2

    DataHead = speech_date_head
    Length_HH = size >> 8
    Length_LL = size & 0x00ff
    Command = 0x01
    EncodingFormat = 0x00

    Data_Pack = [DataHead, Length_HH, Length_LL, Command, EncodingFormat]

    I2C_WriteBytes(Data_Pack)
    I2C_WriteBytes(text)

def TextCtrl(ch, num):
    if num != -1:
        str_T = '[' + ch + str(num) + ']'
        SetBase(str_T)
    else:
        str_T = '[' + ch + ']'
        SetBase(str_T)

# Chip status types
ChipStatus_Type = {
    'ChipStatus_InitSuccessful': 0x4A,  # Initialization successful
    'ChipStatus_CorrectCommand': 0x41,  # Correct command received
    'ChipStatus_ErrorCommand': 0x45,    # Unrecognized command received
    'ChipStatus_Busy': 0x4E,            # Chip busy status
    'ChipStatus_Idle': 0x4F             # Chip idle status
}

def GetChipStatus():
    global i2c_speech_addr
    AskState = [0xfd, 0x00, 0x01, 0x21]
    try:
        I2C_WriteBytes(AskState)
        time.sleep(0.05)
    except:
        print("I2C read/write error")

    try:
        Read_result = bus.read_byte(i2c_speech_addr)
        return Read_result
    except:
        print("I2C read error")

# Synthesis styles
Style_Type = {
    'Style_Single': 0,   # One word at a time
    'Style_Continue': 1  # Continuous synthesis
}

def SetStyle(num):
    TextCtrl('f', num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)   

# Language types
Language_Type = {
    'Language_Auto': 0,       # Auto-detect language
    'Language_Chinese': 1,    # Treats Arabic numbers, units, and special symbols as Chinese
    'Language_English': 2     # Treats Arabic numbers, units, and special symbols as English
}

def SetLanguage(num):
    TextCtrl('g', num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)

# Articulation types
Articulation_Type = {
    'Articulation_Auto': 0,   # Auto-detect word pronunciation
    'Articulation_Letter': 1, # Letter pronunciation
    'Articulation_Word': 2    # Word pronunciation
}

def SetArticulation(num):
    TextCtrl('h', num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)

# Pinyin recognition
Spell_Type = {
    'Spell_Disable': 0, # Disable Pinyin recognition
    'Spell_Enable': 1   # Enable Pinyin recognition
}

def SetSpell(num):
    TextCtrl('i', num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)

# Reader types
Reader_Type = {
    'Reader_XiaoYan': 3,      # Female voice, recommended
    'Reader_XuJiu': 51,       # Male voice, recommended
    'Reader_XuDuo': 52,       # Male voice
    'Reader_XiaoPing': 53,    # Female voice
    'Reader_DonaldDuck': 54,  # Donald Duck voice
    'Reader_XuXiaoBao': 55    # Female child voice
}

def SetReader(num):
    TextCtrl('m', num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)

# Number handling types
NumberHandle_Type = {
    'NumberHandle_Auto': 0,   # Auto-detect
    'NumberHandle_Number': 1, # Handle as a number
    'NumberHandle_Value': 2   # Handle as a value
}

def SetNumberHandle(num):
    TextCtrl('n', num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)

# Zero pronunciation types
ZeroPronunciation_Type = {
    'ZeroPronunciation_Zero': 0, # Pronounce as "zero"
    'ZeroPronunciation_O': 1     # Pronounce as "O"
}

def SetZeroPronunciation(num):
    TextCtrl('o', num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)

# Name pronunciation types
NamePronunciation_Type = {
    'NamePronunciation_Auto': 0,       # Auto-detect surname pronunciation
    'NamePronunciation_Constraint': 1  # Force use of surname pronunciation rules
}

def SetNamePronunciation(num):
    TextCtrl('r', num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)

# Set speech speed (0-10)
def SetSpeed(speed):
    TextCtrl('s', speed)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)

# Set intonation (0-10)
def SetIntonation(intonation):
    TextCtrl('t', intonation)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)

# Set volume (0-10)
def SetVolume(volume):
    TextCtrl('v', volume)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)

# Pronunciation of "1" in numbers
OnePronunciation_Type = {
    'OnePronunciation_Yao': 0, # Pronounce as "Yao"
    'OnePronunciation_Yi': 1   # Pronounce as "Yi"
}

def SetOnePronunciation(num):
    TextCtrl('y', num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)

# Use of rhythm marks "*" and "#"
Rhythm_Type = {
    'Rhythm_Disable': 0, # "*" and "#" are read out as symbols
    'Rhythm_Enable': 1   # "*" indicates a word break, "#" indicates a pause
}

def SetRhythm(num):
    TextCtrl('z', num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)

# Restore default synthesis parameters
def SetRestoreDefault():
    TextCtrl('d', -1)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)


# Define voice control function

In [ ]:
#Write entry
def AsrAddWords(idnum,str):
    global i2c_addr
    global asr_add_word_addr
    words = []
    words.append(asr_add_word_addr)
    words.append(len(str) + 2)
    words.append(idnum)
    for  alond_word in str:
        words.append(ord(alond_word))
    words.append(0)
    print(words)
    for date in words:
        bus.write_byte (i2c_addr, date)
        time.sleep(0.03)

#Set RGB
def RGBSet(R,G,B):
    global i2c_addr
    global asr_rgb_addr
    date = []
    date.append(R)
    date.append(G)
    date.append(B)
    print(date)
    bus.write_i2c_block_data (i2c_addr,asr_rgb_addr,date)

#Read result
def I2CReadByte(reg):
    global i2c_addr
    bus.write_byte (i2c_addr, reg)
    time.sleep(0.05)
    Read_result = bus.read_byte (i2c_addr)
    return Read_result

#Wait busy
def Busy_Wait():
    busy = 255
    while busy != 0:
        busy = I2CReadByte(asr_busy)
        print(asr_busy)	

# "Clear the word entries and module mode data in the power-on buffer. This part needs to be written only during the first use. Subsequently, if no further changes to the settings are required, you can set 1 to 0 or skip this step. Next, set the module's sensitivity and the switch for recognition prompt sound. Afterward, light up the module's RGB light in white for 1 second and sound a beep for 1 second, then announce 'Initialization complete.'"

In [ ]:
'''
The mode and phrase have the function of power-down save, if there is no modification after the first entry, you can change 1 to 0
'''
cleck = 1

if 1:
    bus.write_byte_data(i2c_addr, asr_clear_addr, 0x40)#Clear the power-down buffer area
    Busy_Wait()#Wait for the module to be free
    print("Cache cleared")
    bus.write_byte_data(i2c_addr, asr_mode_addr, 1)
    Busy_Wait()
    print("The mode is set")
    AsrAddWords(0,"xiao ya")
    Busy_Wait()
    AsrAddWords(1,"hong se")
    Busy_Wait()
    AsrAddWords(2,"lv se")
    Busy_Wait()
    AsrAddWords(3,"lan se")
    Busy_Wait()
    AsrAddWords(4,"huang se")
    Busy_Wait()
    AsrAddWords(5,"kai shi fen jian")
    Busy_Wait()
    AsrAddWords(6,"bu yao")
    Busy_Wait()
    AsrAddWords(7,"chong xin pai xu")
    Busy_Wait()
    while cleck != 8:
        cleck = I2CReadByte(asr_num_cleck)
        print(cleck)

bus.write_byte_data(i2c_addr, asr_rec_gain_addr, 0x40)#Set the sensitivity, the recommended value is 0x40-0x55
bus.write_byte_data(i2c_addr, asr_voice_flag, 1)#Set switch sound
bus.write_byte_data(i2c_addr, asr_buzzer, 1)#buzzer
RGBSet(255,255,255)
time.sleep(1)
RGBSet(0,0,0)
bus.write_byte_data(i2c_addr, asr_buzzer, 0)#buzzer

SetReader(Reader_Type["Reader_XiaoPing"])#Select the announcer Xiaoping
SetVolume(8)
Speech_text("Initialization complete",EncodingFormat_Type["GB2312"])
while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
    time.sleep(0.1)  

### Main program

In [ ]:
def camera():
    global color_hsv, model, dp, msg, color_list, color_read

    # Open the camera
    capture = cv.VideoCapture(0)

    # Loop while the camera is open
    while capture.isOpened():
        try:
            result = I2CReadByte(asr_result)
            time.sleep(0.01)
            # Read each frame from the camera
            _, img = capture.read()
            # Resize the image
            img = cv.resize(img, (640, 480))
            xy = [joint1_slider.value, joint2_slider.value]

            if model == 'Calibration':
                time.sleep(0.5)
                _, img = calibration.calibration_map(img, xy, threshold_slider.value)
                model = 'calibration_OK'
            if model == 'calibration_OK':
                time.sleep(0.5)
                try: 
                    write_XYT(XYT_path, xy, threshold_slider.value)
                except Exception: 
                    print("File XYT_config Error !!!")
                dp, img = calibration.calibration_map(img, xy, threshold_slider.value)
                model = "Detection2"
            if len(dp) != 0: 
                img = calibration.Perspective_transform(dp, img)
            if model == 'calibration_Cancel':  
                dp = []
                msg = {}
                color_read = 0
                model = "General"
            if model == 'Detection2':
                _, img = capture.read()
                # Resize the image
                img = cv.resize(img, (640, 480))
                xy = [joint1_slider.value, joint2_slider.value]
                color_read = 0
                color_list['1'] = "red"
                color_list['2'] = "green"
                color_list['3'] = "blue"
                color_list['4'] = "yellow"
                img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                time.sleep(0.1)
                Speech_text("识别到", EncodingFormat_Type["GB2312"])
                while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:  # Wait for the current speech to finish
                    time.sleep(0.1)  
                if coo1 == 'red':
                    color_read += 1
                    Speech_text("红色", EncodingFormat_Type["GB2312"])
                    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:  # Wait for the current speech to finish
                        time.sleep(0.1)   
                if coo2 == 'green':
                    color_read += 1
                    Speech_text("绿色", EncodingFormat_Type["GB2312"])
                    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:  # Wait for the current speech to finish
                        time.sleep(0.1)  
                if coo3 == 'blue':
                    color_read += 1
                    Speech_text("蓝色", EncodingFormat_Type["GB2312"])
                    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:  # Wait for the current speech to finish
                        time.sleep(0.1)  
                if coo4 == 'yellow':
                    color_read += 1
                    Speech_text("黄色", EncodingFormat_Type["GB2312"])
                    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:  # Wait for the current speech to finish
                        time.sleep(0.1)  
                imgbox.value = cv.imencode('.jpg', img)[1].tobytes()
                Speech_text("第一个要排序的颜色是：", EncodingFormat_Type["GB2312"])
                while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:  # Wait for the current speech to finish
                    time.sleep(0.1)  
                model = "list1"
                    
            if model == 'list1' and color_read > 0:  
                if result == 1:
                    color_list['1'] = "red" 
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist1"
                elif result == 2:
                    color_list['1'] = "green"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist1"
                elif result == 3:
                    color_list['1'] = "blue"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist1"
                elif result == 4:
                    color_list['1'] = "yellow"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist1"
                elif result == 6:
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist1"
            if model == 'tolist1':
                if color_read > 0:
                    Speech_text("好的，第二个要排序的颜色是：", EncodingFormat_Type["GB2312"])
                    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:  # Wait for the current speech to finish
                        time.sleep(0.1) 
                    model = "list2"
                elif color_read == 0:
                    Speech_text("好的，排序完成，是否开始分拣", EncodingFormat_Type["GB2312"])
                    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:  # Wait for the current speech to finish
                        time.sleep(0.1) 
                    model = "toGrap"
                   
            if model == 'list2' and color_read > 0:  
                if result == 1:
                    color_list['2'] = "red" 
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist2"
                elif result == 2:
                    color_list['2'] = "green"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist2"
                elif result == 3:
                    color_list['2'] = "blue"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist2"
                elif result == 4:
                    color_list['2'] = "yellow"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist2"
                elif result == 6:
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist2"
            if model == 'tolist2':
                if color_read > 0:
                    Speech_text("好的，第三个要排序的颜色是：", EncodingFormat_Type["GB2312"])
                    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:  # Wait for the current speech to finish
                        time.sleep(0.1) 
                    model = "list3"
                elif color_read == 0:
                    Speech_text("好的，排序完成，是否开始分拣", EncodingFormat_Type["GB2312"])
                    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:  # Wait for the current speech to finish
                        time.sleep(0.1) 
                    model = "toGrap"
                         
            if model == 'list3' and color_read > 0:  
                if result == 1:
                    color_list['3'] = "red" 
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist3"
                elif result == 2:
                    color_list['3'] = "green"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist3"
                elif result == 3:
                    color_list['3'] = "blue"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist3"
                elif result == 4:
                    color_list['3'] = "yellow"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist3"
                elif result == 6:
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist3"
            if model == 'tolist3':
                if color_read > 0:
                    Speech_text("好的，第四个要排序的颜色是：", EncodingFormat_Type["GB2312"])
                    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:  # Wait for the current speech to finish
                        time.sleep(0.1) 
                    model = "list4"
                elif color_read == 0:
                    Speech_text("好的，排序完成，是否开始分拣", EncodingFormat_Type["GB2312"])
                    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:  # Wait for the current speech to finish
                        time.sleep(0.1) 
                    model = "toGrap"
                             
            if model == 'list4' and color_read > 0:  
                if result == 1:
                    color_list['4'] = "red" 
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist4"
                elif result == 2:
                    color_list['4'] = "green"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist4"
                elif result == 3:
                    color_list['4'] = "blue"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist4"
                elif result == 4:
                    color_list['4'] = "yellow"
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist4"
                elif result == 6:
                    img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                    result = 0
                    color_read -= 1
                    model = "tolist4"
            if model == 'tolist4':
                Speech_text("好的，排序完成，是否开始分拣", EncodingFormat_Type["GB2312"])
                while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:  # Wait for the current speech to finish
                    time.sleep(0.1) 
                model = "toGrap"
                    
            if model == 'toGrap':  
                img, msg, coo1, coo2, coo3, coo4 = target.select_color(img, color_hsv, color_list)
                if result == 5:
                    result = 0
                    model = "Grap"

            if result == 7:
                result = 0
                msg = {}
                color_list = {}
                model = "Detection2"
                Speech_text("好的", EncodingFormat_Type["GB2312"])
                while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:  # Wait for the current speech to finish
                    time.sleep(0.1) 
            if model == "Reset_list":
                msg = {}
                color_list = {}
                model = "General"
            if len(msg) != 0 and model == 'Grap':
                threading.Thread(target=target.target_run, args=(msg, xy)).start()
                msg = {}
                model = "General"
            if model == 'Exit':
                cv.destroyAllWindows()
                capture.release()
                break
            imgbox.value = cv.imencode('.jpg', img)[1].tobytes()
        except KeyboardInterrupt:
            capture.release()


### Start

In [ ]:
# display(controls_box)
model = 'Calibration'
display(controls_box,output)
threading.Thread(target=camera, ).start()